<h1>Create and Populate PostgreSQL Output Database for FVS Simulations</h1>

In [1]:
# import all the SQL query strings for creating FVS output tables
from FVSoutput_SQL_createDBtables import *

In [ ]:
fvs_cases

In [2]:
import psycopg2
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT

## Create FVSOut

In [ ]:
# enter the name and username/owner for the FVS output databse you want as strings
db_out = '"Rotations"'
owner = 'ubuntu'

In [ ]:
# enter the dbname, user, and host for creating a new database within PostgreSQL (e.g., your default postgres user and db)
my_db = 'postgres'
my_user = 'postgres'
my_host = 'localhost'

conn_string = "dbname={dbname} user={user} host={host}".format(dbname=my_db, user=my_user, host=my_host)
conn = psycopg2.connect(conn_string) # password stored in pgpass
conn.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)

with conn:
    with conn.cursor() as cur:
        # check to see if the owner exists as a user already, if not, create it
        SQL = "SELECT rolname FROM pg_roles WHERE rolname=(%s)"
        cur.execute(SQL, [owner])
        if cur.fetchone() == None: # user doesn't exist
            print(owner, "doesn't exist as a PostgreSQL user. Creating a new user.", end='... ')
            cur.execute('CREATE USER {username};'.format(username=owner))
            print('Done.')
        
        # check to see if database exists
        SQL = "SELECT datname FROM pg_database WHERE datname={dbname};".format(dbname=db_out)
        try:
            cur.execute(SQL)
            print('That database already exists... handle your business.')
        except psycopg2.ProgrammingError: # if database doesn't exist, programming error is returned
            cur.execute('CREATE DATABASE {dbname} OWNER={owner};'.format(dbname=db_out, owner=owner))

    with conn.cursor() as cur:
        # confirm the database is created
        cur.execute(SQL, [db_out])
        print('Created your new database:', db_out)

conn.close()

In [ ]:
# enter the dbname, user, and host for creating a new database within PostgreSQL (e.g., your default postgres user and db)
my_db = 'postgres'
my_user = 'postgres'
my_host = 'localhost'

conn_string = "dbname={dbname} user={user} host={host}".format(dbname=my_db, user=my_user, host=my_host)
conn = psycopg2.connect(conn_string) # password stored in pgpass
with conn:
    with conn.cursor() as cur:
       # check to see if database exists
        SQL = "SELECT datname FROM pg_database WHERE datname={dbname};".format(dbname=db_out)
        cur.execute(SQL)
        print(cur.fetchall())
conn.close()

A helper function to choose what tables to create in FVS Output database.

In [3]:
def create_tables(conn_str, table_SQLs):
    '''
    Creates tables in a PostgreSQL database to hold FVS Outputs.
    ===========
    Arguments:
    conn_str = A string for the database connection. 
    table_SQLs = A list of valid FVS table names (imported from FVSoutput_SQL_createDBtables.py)
    '''
    with psycopg2.connect(conn_string) as conn:
        with conn.cursor() as cur:
            for SQL in table_SQLs:
                print(SQL)
                cur.execute(SQL)
                print('Created', SQL.split(' ')[2], end='... ')
    conn.close()
    print('Done.')

## Create Tables in FVSOut

The following tables are available to use. All should be pre-prended with "fvs_": 

**Base FVS (incl. DB Extension):**  
atrtrlist, cases, compute, cutlist, strclass, summary, treelist

**Fire & Fuels Extension (FFE):**  
*General:* canprofile, snagdet, down_wood_cov, down_wood_vol, mortality, snagsum  
*Fire/Fuels:* burnreport, consumption, potfire, fuels  
*Carbon:* carbon, hrv_carbon, 



**ECON Extension:**  
econharvestvalue, econsummary

**Mountain Pine Beetle (MPB) Impact Model:**  
bm_bkp, bm_main, bm_tree, bm_vol

**Climate-FVS:**  
climate  

**Dwarf Mistletoe:**  
dm_spp_sum, dm_stnd_sum  

**Western Root Disease:**  
rd_sum, rd_det, rd_beetle

In [4]:
# specify the tables you want to create
my_tables = [fvs_cases, fvs_summary, fvs_carbon, fvs_hrv_carbon, fvs_econharvestvalue, fvs_econsummary]

## Create the tables in your output database

In [5]:
conn_string = "dbname={dbname} user={user} host={host}".format(dbname="FVSOut", user='postgres', host='localhost')
create_tables(conn_string, my_tables)


CREATE TABLE cases (
caseid varchar(36) NOT NULL,
stand_cn varchar(40),
standid varchar(26),
mgmtid varchar(4),
runtitle varchar(72),
keywordfile varchar(50),
samplingwt real,
variant varchar(2),
version varchar(10),
rv varchar(8),
groups varchar(250),
rundatetime varchar(19)
);

CREATE VIEW fvs_cases AS
SELECT * FROM cases LIMIT 0;

CREATE RULE redirect_cases AS
ON INSERT TO fvs_cases
DO INSTEAD INSERT INTO cases VALUES (NEW.*);

Created cases... 
CREATE TABLE summary (
caseid varchar(36),
standid varchar(26),
year integer,
age integer,
tpa real,
ba real,
sdi real,
ccf real,
topht real,
qmd real,
tcuft real,
mcuft real,
bdft real,
rtpa real,
rtcuft real,
rmcuft real,
rbdft real,
atba real,
atsdi real,
atccf real,
attopht real,
atqmd real,
prdlen integer,
acc real,
mort real,
mai real,
fortyp integer,
sizecls integer,
stkcls integer
);

CREATE VIEW fvs_summary AS
SELECT * FROM summary LIMIT 0;

CREATE RULE redirect_summary AS
ON INSERT TO fvs_summary
DO INSTEAD INSERT INTO summary VAL